In [1]:
class Material:
    def __init__(self, titulo, estado="disponible"):
        self.titulo = titulo
        self.estado = estado

In [2]:
class Libro(Material):
    def __init__(self, titulo, autor, genero, estado="disponible"):
        super().__init__(titulo, estado)
        self.autor = autor
        self.genero = genero

In [4]:
class Revista(Material):
    def __init__(self, titulo, edicion, periodicidad, estado="disponible"):
        super().__init__(titulo, estado)
        self.edicion = edicion
        self.periodicidad = periodicidad


In [5]:
class MaterialDigital(Material):
    def __init__(self, titulo, tipo_archivo, enlace):
        super().__init__(titulo, "disponible")
        self.tipo_archivo = tipo_archivo
        self.enlace = enlace

In [6]:
class Persona:
    def __init__(self, nombre):
        self.nombre = nombre


In [7]:
class Usuario(Persona):
    def __init__(self, nombre):
        super().__init__(nombre)
        self.prestamos = []
        self.penalizaciones = []

    def consultar_catalogo(self, catalogo):
        return catalogo.listar_materiales()

    def devolver_material(self, prestamo, fecha_devolucion_real):
        if fecha_devolucion_real > prestamo.fecha_devolucion:
            dias_retraso = (fecha_devolucion_real - prestamo.fecha_devolucion).days
            penalizacion = Penalizacion(self, dias_retraso)
            self.penalizaciones.append(penalizacion)
        prestamo.material.estado = "disponible"
        self.prestamos.remove(prestamo)


In [9]:
class Bibliotecario(Persona):
    def __init__(self, nombre):
        super().__init__(nombre)

    def agregar_material(self, catalogo, material):
        catalogo.agregar_material(material)

    def gestionar_prestamo(self, prestamo):
        prestamo.material.estado = "prestado"
        prestamo.usuario.prestamos.append(prestamo)

    def transferir_material(self, material, sucursal_origen, sucursal_destino):
        if material in sucursal_origen.catalogo:
            sucursal_origen.catalogo.remove(material)
            sucursal_destino.catalogo.append(material)


In [8]:
class Sucursal:
    def __init__(self, nombre):
        self.nombre = nombre
        self.catalogo = []

    def agregar_material(self, material):
        self.catalogo.append(material)


In [10]:
class Prestamo:
    def __init__(self, usuario, material, fecha_prestamo, fecha_devolucion):
        self.usuario = usuario
        self.material = material
        self.fecha_prestamo = fecha_prestamo
        self.fecha_devolucion = fecha_devolucion

In [11]:
class Penalizacion:
    def __init__(self, usuario, dias_retraso, multa_por_dia=5):
        self.usuario = usuario
        self.dias_retraso = dias_retraso
        self.multa_total = dias_retraso * multa_por_dia


In [12]:
class Catalogo:
    def __init__(self):
        self.materiales = []

    def agregar_material(self, material):
        self.materiales.append(material)

    def listar_materiales(self):
        return [material.titulo for material in self.materiales]

    def buscar_por_criterio(self, criterio, valor):
        return [material for material in self.materiales if getattr(material, criterio, None) == valor]


In [13]:
# Ejemplo de uso:
from datetime import date, timedelta

catalogo = Catalogo()
bibliotecario = Bibliotecario("Ana")
usuario = Usuario("Carlos")

sucursal_a = Sucursal("Sucursal Centro")
sucursal_b = Sucursal("Sucursal Norte")

libro = Libro("1984", "George Orwell", "Distopía")
revista = Revista("National Geographic", "Edición 202", "Mensual")
material_digital = MaterialDigital("Curso Python", "PDF", "www.ejemplo.com")

bibliotecario.agregar_material(catalogo, libro)
bibliotecario.agregar_material(catalogo, revista)
bibliotecario.agregar_material(catalogo, material_digital)

sucursal_a.agregar_material(libro)
sucursal_a.agregar_material(revista)
sucursal_b.agregar_material(material_digital)

print("Materiales en el catálogo:", usuario.consultar_catalogo(catalogo))

prestamo = Prestamo(usuario, libro, date(2024, 2, 1), date(2024, 2, 15))
bibliotecario.gestionar_prestamo(prestamo)

print(f"{usuario.nombre} ha tomado prestado: {[p.material.titulo for p in usuario.prestamos]}")

# Simulación de devolución con retraso
fecha_devolucion_real = date(2024, 2, 20)
usuario.devolver_material(prestamo, fecha_devolucion_real)
print(f"{usuario.nombre} ha recibido una penalización de: {usuario.penalizaciones[0].multa_total} unidades monetarias")

# Transferencia de material entre sucursales
bibliotecario.transferir_material(libro, sucursal_a, sucursal_b)
print(f"El libro '1984' ha sido transferido a {sucursal_b.nombre}")


Materiales en el catálogo: ['1984', 'National Geographic', 'Curso Python']
Carlos ha tomado prestado: ['1984']
Carlos ha recibido una penalización de: 25 unidades monetarias
El libro '1984' ha sido transferido a Sucursal Norte
